気になっていること
* PositiveとNegativeは実際どれくらい離れているのか
* 特徴量は縮退していないか
* 一部の次元だけで判断されていないか =>これはCMD的にも多分発生してはいなそうではある


手法アイディア
* ランダム行列を挟んだ上で、複数の重みでスコアを出す
* Maximum Classifier Discrepancyみたいなことをする
* Hard Negative以外は無視する（Positiveの最小スコアより低いNegativeについては信用しない） => 学習が遅くなりそう（初期で全く学習がされない？）

In [1]:
from run_sacred import data_ingredient, method_ingredient, optim_ingredient
from sacred import Experiment
ex = Experiment('jupyter_ex', ingredients=[data_ingredient, method_ingredient, optim_ingredient], interactive=True)

In [2]:
@ex.main
def main(_config):
    print(_config)
    return _config

In [3]:
ex.run(config_updates={'seed':123456})

WARNING - jupyter_ex - No observers have been added to this run
INFO - jupyter_ex - Running command 'main'
INFO - jupyter_ex - Started
INFO - jupyter_ex - Result: {'seed': 123456, 'dataset': {'name': 'oppG', 'validation': 'ADL4-ADL5', 'test_domain': 'S1', 'L': 12, 'K': 5}, 'method': {'name': 'CPC', 'hidden': 1600, 'context': 800, 'num_gru': 1, 'sampler_mode': 'random'}, 'optim': {'lr': 0.0001, 'num_batch': 10000, 'batch_size': 128}}
INFO - jupyter_ex - Completed after 0:00:00


{'seed': 123456, 'dataset': {'name': 'oppG', 'validation': 'ADL4-ADL5', 'test_domain': 'S1', 'L': 12, 'K': 5}, 'method': {'name': 'CPC', 'hidden': 1600, 'context': 800, 'num_gru': 1, 'sampler_mode': 'random'}, 'optim': {'lr': 0.0001, 'num_batch': 10000, 'batch_size': 128}}


In [4]:
_config = {'seed': 123456, 'dataset': {'name': 'oppG', 'validation': 'ADL4-ADL5', 'test_domain': 'S1', 'L': 12, 'K': 5}, 'method': {'name': 'CPC', 'hidden': 1600, 'context': 800, 'num_gru': 1, 'sampler_mode': 'random'}, 'optim': {'lr': 0.0001, 'num_batch': 10000, 'batch_size': 128}}

In [5]:
from run_sacred import *
from sacred_wrap import MongoExtractor
from utils import flatten_dict
from copy import deepcopy
import os
import torch

In [6]:
datasets = get_dataset(**_config['dataset'])
train_dataset_joint, valid_dataset_joint, train_dataset_marginal, valid_dataset_marginal, test_dataset = datasets


In [7]:
# Load pretrained model
model = get_model(input_shape=train_dataset_joint.get('input_shape'), K=_config['dataset']['K'], **_config['method'])
query = deepcopy(_config)
query = flatten_dict(query)
extractor = MongoExtractor(None, 'CPC_test')
result = list(extractor.find(query, ['config', 'info'], False, 'COMPLETED'))
assert len(result) == 1, "There are too many or no results. Please check the query {}".format(query)
result = result[0]

path = os.path.join(result['info']['log_dir'], 'model_{}.pth'.format(_config['optim']['num_batch']))
model = model.cpu()
model.load_state_dict(torch.load(path, map_location='cpu'))
model = model.cuda()

1600 800


In [8]:
model

CPCModel(
  (g_enc): Encoder(
    (feature): Sequential(
      (0): Conv2d(1, 50, kernel_size=(1, 5), stride=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(50, 40, kernel_size=(1, 5), stride=(1, 1))
      (4): ReLU()
      (5): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(40, 20, kernel_size=(1, 3), stride=(1, 1))
      (7): ReLU()
      (8): Dropout(p=0.5)
      (9): Flatten()
      (10): Linear(in_features=4520, out_features=1600, bias=True)
      (11): ReLU()
      (12): Dropout(p=0.5)
    )
  )
  (c_enc): ContextEncoder(
    (gru): GRU(1600, 800)
  )
  (predictor): Predictor(
    (linears): ModuleList(
      (0): Linear(in_features=800, out_features=1600, bias=True)
      (1): Linear(in_features=800, out_features=1600, bias=True)
      (2): Linear(in_features=800, out_features=1600, bias=True)
      (3): Linear(in_features=800, out_f

In [9]:
train_loader_joint = data.DataLoader(
    train_dataset_joint, batch_size=_config['optim']['batch_size'], shuffle=True)
train_loader_marginal = data.DataLoader(
    train_dataset_marginal, batch_size=_config['optim']['batch_size'], shuffle=True)


In [10]:
"""
optimizer = optim.Adam(model.parameters(), lr=_config['optim']['lr'])
for num_iter in range(100):
    loss = train_CPC(train_loader_joint, train_loader_marginal, model, optimizer)
"""

"\noptimizer = optim.Adam(model.parameters(), lr=_config['optim']['lr'])\nfor num_iter in range(100):\n    loss = train_CPC(train_loader_joint, train_loader_marginal, model, optimizer)\n"

In [21]:
def validate(dataset_joint, dataset_marginal, model, num_eval=10, batch_size=128):
    """Evaluate the model."""
    model.eval()

    loader_joint = data.DataLoader(dataset_joint, batch_size=batch_size, shuffle=True, drop_last=True)
    loader_marginal = data.DataLoader(dataset_marginal, batch_size=batch_size, shuffle=True, drop_last=True)

    if num_eval is None:
        num_eval = len(loader_joint)

    K = dataset_marginal.T
    L = dataset_joint.T - K
    losses = [0] * K
    TP = [0] * K
    TN = [0] * K
    FP = [0] * K
    FN = [0] * K

    for i, ((X_j, _), (X_m, _)) in enumerate(zip(loader_joint, loader_marginal)):
        X_j = X_j.float().cuda()
        X_m = X_m.float().cuda()

        score_j_list, score_m_list = model(X_j, X_m, L, K)
        for k in range(K):
            losses[k] += (-1.0 * torch.log(torch.clamp(score_j_list[k], min=1e-8)).mean()
                          - torch.log(torch.clamp(1-score_m_list[k], min=1e-8)).mean()).item()
            TP[k] += (score_j_list[k] > 0.5).sum().item()
            TN[k] += (score_m_list[k] < 0.5).sum().item()
            FP[k] += (score_m_list[k] > 0.5).sum().item()
            FN[k] += (score_j_list[k] < 0.5).sum().item()
        if i+1 == num_eval:
            break
    results = OrderedDict()
    for i in range(K):
        print(score_j_list[i].min(), score_m_list[i].max())

    for k in range(K):
        results['loss-{}'.format(k)] = losses[k] / (2*(i+1))
        results['accuracy-{}'.format(k)] = float(TP[k]+TN[k]) / float(FP[k]+FN[k]+TP[k]+TN[k])

    model.train()
    return results

In [22]:
train_result = validate(train_dataset_joint, train_dataset_marginal, model, num_eval=10)

tensor(0.9789, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.6057, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.9750, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.1787, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.9845, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.4894, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.9694, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.6261, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.9707, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.6963, device='cuda:0', grad_fn=<MaxBackward1>)


In [26]:
from cpc import get_context

X_j, _ = train_loader_joint.__iter__().__next__()
X_m, _ = train_loader_marginal.__iter__().__next__()
X_j = X_j.float().cuda()
X_m = X_m.float().cuda()
K = train_dataset_marginal.T
L = train_dataset_joint.T - K

c = get_context(X_j[..., :L], model.g_enc, model.c_enc)
score_j = [None] * K
score_m = [None] * K
for i in range(K):
    z_j = model.g_enc(X_j[..., L+i])
    z_m = model.g_enc(X_m[..., i])
    z_p = model.predictor(c, i)
    score_j[i] = torch.sigmoid(torch.bmm(z_j.unsqueeze(1), z_p.unsqueeze(2)).squeeze(2))
    score_m[i] = torch.sigmoid(torch.bmm(z_m.unsqueeze(1), z_p.unsqueeze(2)).squeeze(2))


In [30]:
L = _config['dataset']['L']
K = _config['dataset']['K']
c = model.get_context(X_j[..., :L])
predictions = model.get_predictions(c, K)


In [34]:
X_m = [X_m, X_m]

In [37]:
if isinstance(X_m, list):
    score_m = [None] * len(X_m)
    for i, X in enumerate(X_m):
        score_m[i] = model.get_score_of(X, K, predictions)

In [41]:
isinstance(score_m[0], list)

True

* 対照推定における基本統計量だけしか学ばれない問題 => Dropoutで解決可能では？

In [43]:
train_loader_joint.__iter__().__next__()

[tensor([[[[[-1.3281e+00, -1.3271e+00, -1.3261e+00,  ..., -1.3144e+00,
             -1.3134e+00, -1.3124e+00],
            [-1.3280e+00, -1.3270e+00, -1.3261e+00,  ..., -1.3143e+00,
             -1.3133e+00, -1.3124e+00],
            [-1.3279e+00, -1.3270e+00, -1.3260e+00,  ..., -1.3143e+00,
             -1.3133e+00, -1.3123e+00],
            ...,
            [-1.3263e+00, -1.3253e+00, -1.3244e+00,  ..., -1.3126e+00,
             -1.3116e+00, -1.3107e+00],
            [-1.3263e+00, -1.3253e+00, -1.3243e+00,  ..., -1.3126e+00,
             -1.3116e+00, -1.3106e+00],
            [-1.3262e+00, -1.3252e+00, -1.3242e+00,  ..., -1.3125e+00,
             -1.3115e+00, -1.3105e+00]],
 
           [[-2.5911e-01, -1.8645e-01, -2.6315e-01,  ...,  2.6567e-01,
              2.2933e-01,  2.8989e-01],
            [-3.7618e-01,  1.9424e-02,  5.9791e-02,  ..., -1.0463e+00,
              2.8585e-01,  2.3741e-01],
            [-3.7214e-01, -3.1966e-01, -1.0168e-01,  ..., -6.5875e-01,
              5.5631e